In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid
import pandas as pd
import datetime

def GetParamIfExist(param, params_dict):
    if param in params_dict.keys():
        return params_dict[param]
    else:
        return None

def SaveResults(datasetID, bestScore, bestParams, clfName, model):
    path = "results/batch/dynamic/{}-{}.csv".format(model, clfName)
    cols = ['model', 'K', 'excludingPercentage', 'clfName', 'distanceMetric', 
            'beta', 'bestMacrof1', 'date']
    
    try:
        results = pd.read_csv(path, index_col=0)
    except FileNotFoundError:
        results = pd.DataFrame(columns = cols)
    
    if (datasetID in  results.index):
        results.loc[datasetID, cols[0]] = model
        results.loc[datasetID, cols[1]] = GetParamIfExist(cols[1], bestParams)
        results.loc[datasetID, cols[2]] = GetParamIfExist(cols[2], bestParams)
        results.loc[datasetID, cols[3]] = GetParamIfExist(cols[3], bestParams)
        results.loc[datasetID, cols[4]] = GetParamIfExist(cols[4], bestParams)
        results.loc[datasetID, cols[5]] = GetParamIfExist(cols[5], bestParams)
        results.loc[datasetID, cols[6]] = bestScore
        results.loc[datasetID, cols[7]] = str(datetime.datetime.now())
    else:
        result = pd.DataFrame([[model,
                                GetParamIfExist(cols[1], bestParams),
                                GetParamIfExist(cols[2], bestParams),
                                GetParamIfExist(cols[3], bestParams),
                                GetParamIfExist(cols[4], bestParams),
                                GetParamIfExist(cols[5], bestParams),
                                bestScore,
                                str(datetime.datetime.now())]],
                                columns=cols, index = [datasetID])
        results = results.append(result)
        
    results.to_csv(path)

'''
def writeInfo(info, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_20PERCENT-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_20PERCENT-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_20PERCENT-{}.txt".format(distanceMetric,clfName)
    file = open(path,"a") 
    file.write(info)
    file.close() 
'''
def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main(distanceMetric, beta=None):
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    if beta != None and distanceMetric == 'BBD':
        name = 'amandadcp-bbd{}'.format(beta)
    else:
        name = 'amandadcp-{}'.format(distanceMetric)
    
    #loading datasets
    
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadKeystroke, setup.loadNOAADataset]
    
    #datasets = [setup.loadElec2Data]
    #datasets = [setup.loadNOAA2Dataset]
    
    #arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    arrClfName = ['LP']
    if (distanceMetric == 'BBD') and (beta is None):
        raise Exception(("AMANDA-DCP with BBD must have a beta value. beta = {}").format(str(beta)))
    
    if beta is list:
        allBetas = True
    else:
        allBetas=False
    
    #writeInfo("**************** GRID SEARCH - 20% DOS DADOS ****************", distanceMetric, beta)
    
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))

        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            splitPercentage = 0.05
            batches=10
            
            if i==len(datasets)-2: #NOAA
                batches=8
            elif i==len(datasets)-1: #Keystroke
                batches=4

            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(splitPercentage*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]
            
            initialLabeledData = int((1/batches)*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/(batches - 1))

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches-1], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":[beta]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except:
                    print("An error occured in ", description, g)
                    #raise

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            #writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
            SaveResults(description, finalScore, best_grid, clfName, name)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    distanceMetric = 'BBD'
    #betas = [-50, -1, -0.5, -0.1, 1.001, 1.1, 2, 3, 5, 10, 50]
    betas = [-1]
    
    # Grid search for each beta value - BBD
    for beta in betas:
        main(distanceMetric, beta)
        print('\n\n\n')
        
    # grid search for Hellinger
    distanceMetric = 'Hellinger'
    main(distanceMetric, beta)
    print('\n\n\n')
    
    distanceMetric = 'Hellinger2'
    main(distanceMetric, beta)
    print('\n\n\n')


**************** BEGIN of LP-BBD-1.000 results ****************
One Class Diagonal Translation. 2 Dimensional data.: 10 batches of 80 instances
0.9623094764080794 {'K': 2, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9665248288482002 {'K': 3, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.96792002272062 {'K': 4, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9748490641253648 {'K': 5, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.9735123137072568 {'K': 6, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': T

0.7123381397093941 {'K': 8, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.7700322892023781 {'K': 9, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.7620177955739068 {'K': 10, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.7411166009245456 {'K': 11, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.6833773649936508 {'K': 12, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
0.8532238894426263
{'K': 4, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', '

0.8636916749439215 {'K': 4, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 120, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 120}
0.7947613398356816 {'K': 5, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 120, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 120}
0.8253274986506217 {'K': 6, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 120, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 120}
0.7771732479107678 {'K': 7, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 120, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 120}
0.7316122698535142 {'K': 8, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 120, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 120}
0.71391493049833 {'K': 9, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BB

0.9971086406087425 {'K': 10, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 500, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 500}
0.9971086406087425 {'K': 11, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 500, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 500}
0.9971086406087425 {'K': 12, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 500, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 500}
0.9973308690532972
{'K': 2, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 500, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 500}
Two Bidimensional Multimodal Gaussian Classes.: 10 batches of 1000 instances
0.9978884413417456 {'K': 2, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.997888

0.9993331321606421 {'K': 4, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9991108332396887 {'K': 5, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9992220018229278 {'K': 6, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9992220018229278 {'K': 7, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9992220018229278 {'K': 8, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.999333137939665 {'K': 9, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceM

...

0.9791875649622802 {'K': 9, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 625, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 625}
0.9805945913166302 {'K': 10, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 625, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 625}
0.9807879621919798 {'K': 11, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 625, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 625}
0.9825432017450226 {'K': 12, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 625, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 625}
0.9825432017450226
{'K': 12, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 625, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 625}
Four Classes Rotating with Expansion V2. Bidimensio

Four Classes Expanding and One Class Fixed. Bidimensional.: 10 batches of 866 instances
0.9563343540965457 {'K': 2, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 866, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 866}
0.9591214830557847 {'K': 3, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 866, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 866}
0.9627281310087987 {'K': 4, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 866, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 866}
0.9619143615615161 {'K': 5, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 866, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 866}
0.9586982165010904 {'K': 6, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 866, 'isBatchMode': True, 'pool

0.871099188906705 {'K': 6, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.8630614784477966 {'K': 7, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.871706765382415 {'K': 8, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.867967802220581 {'K': 9, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.8677480797865527 {'K': 10, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.8694599539350567 {'K': 11, 'batches': 9, 'bet

0.9751996482975677 {'K': 11, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9787614722205605 {'K': 12, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.9796549002810527
{'K': 5, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
Rotated checkerboard dataset. Rotating 2*PI.: 10 batches of 1000 instances
0.6511405442066902 {'K': 2, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1000}
0.6334103128615537 {'K': 3, 'batches': 9, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1000, 'isBatchMode': True, 'poolSi

0.4413143215399464 {'K': 6, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.4745586520358985 {'K': 7, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.4819778471072234 {'K': 8, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.4727291878165052 {'K': 9, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.47372199376128427 {'K': 10, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 226, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 227}
0.47103158828835445 {'K': 11, 'batches': 3, 'beta': -

# 